In [ ]:
import requests
import os
import time
import math

# --- 配置参数 ---
IMG_SIZE = 'large' # 图片尺寸 ('square', 'small', 'medium', 'large', 'original')
PER_PAGE = 200      # 每次 API 请求获取的观察记录数量 (API 最大值为 200)
MAX_IMAGES_TO_DOWNLOAD = 2000 # 你想要下载的最大图片数量 (设置为 None 则下载所有)

# --- 辅助函数：下载单个图片 ---
def download_image(url, filepath):
    """下载单个图片并保存到指定路径"""
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()  # 如果请求失败则引发 HTTPError

        with open(filepath, 'wb') as f:
            for chunk in response.iter_content(1024):
                f.write(chunk)
        # print(f"  成功下载: {os.path.basename(filepath)}")
        return True
    except requests.exceptions.RequestException as e:
        print(f"  下载失败 {url}: {e}")
        return False
    except IOError as e:
        print(f"  保存文件失败 {filepath}: {e}")
        return False

# --- 主要函数：获取观察记录并下载图片 ---
def download_inaturalist_images_by_taxon(taxon_id, output_dir):
    """
    根据 taxon_id 从 iNaturalist 下载图片。

    参数:
        taxon_id (int): 要下载图片的 iNaturalist 分类单元 ID。
        output_dir (str): 保存下载图片的目录路径。
    """
    print(f"开始为 Taxon ID: {taxon_id} 下载图片...")
    print(f"图片将保存到: {output_dir}")
    print(f"图片尺寸: {IMG_SIZE}")
    if MAX_IMAGES_TO_DOWNLOAD:
        print(f"最多下载: {MAX_IMAGES_TO_DOWNLOAD} 张图片")
    else:
        print("尝试下载所有可用的图片。")
    print("-" * 30)

    # 1. 创建输出目录（如果不存在）
    if not os.path.exists(output_dir):
        try:
            os.makedirs(output_dir)
            print(f"创建目录: {output_dir}")
        except OSError as e:
            print(f"错误：无法创建目录 {output_dir}: {e}")
            return

    # 2. 循环获取 API 页面并下载
    page = 1
    downloaded_count = 0
    total_results = None # 用于存储 API 返回的总结果数

    while True:
        # 检查是否已达到下载上限
        if MAX_IMAGES_TO_DOWNLOAD is not None and downloaded_count >= MAX_IMAGES_TO_DOWNLOAD:
            print(f"\n已达到下载上限 ({MAX_IMAGES_TO_DOWNLOAD} 张图片)。")
            break

        # 构建 API 请求 URL
        api_url = "https://api.inaturalist.org/v1/observations"
        params = {
            'taxon_id': taxon_id,
            'photos': 'true',       # 只获取带照片的观察记录
            'per_page': PER_PAGE,
            'page': page,
            'order': 'desc',        # 可以按需更改排序，例如 'desc', 'asc'
            'order_by': 'created_at' # 按创建时间排序
        }

        print(f"\n正在获取第 {page} 页数据...")

        try:
            response = requests.get(api_url, params=params)
            response.raise_for_status() # 检查请求是否成功
            data = response.json()

        except requests.exceptions.RequestException as e:
            print(f"  API 请求错误: {e}")
            print("  等待一段时间后重试...")
            time.sleep(REQUEST_DELAY * 5) # 出错时等待更长时间
            continue # 跳过当前页，尝试下一轮循环 (或考虑更复杂的重试逻辑)
        except ValueError as e: # JSON 解码错误
             print(f"  无法解析 API 响应 (非 JSON): {e}")
             print(f"  响应内容: {response.text[:200]}...") # 显示部分响应内容以帮助调试
             print("  跳过此页。")
             time.sleep(REQUEST_DELAY)
             page += 1
             continue


        results = data.get('results', [])
        if not results:
            print("  当前页面没有更多结果。")
            if total_results is None and page == 1:
                 print("  未找到该 Taxon ID 的任何带照片的观察记录。")
            break # 结束循环

        # 更新总结果数 (仅在第一页执行)
        if page == 1:
            total_results = data.get('total_results', 0)
            print(f"  找到总计约 {total_results} 条带照片的观察记录。")
            if MAX_IMAGES_TO_DOWNLOAD:
                total_pages = math.ceil(min(total_results, MAX_IMAGES_TO_DOWNLOAD) / PER_PAGE)
            else:
                 total_pages = math.ceil(total_results / PER_PAGE)
            print(f"  预计需要获取 {total_pages} 页数据。")


        print(f"  处理 {len(results)} 条观察记录...")

        # 3. 遍历观察记录并下载图片
        for obs in results:
             if MAX_IMAGES_TO_DOWNLOAD is not None and downloaded_count >= MAX_IMAGES_TO_DOWNLOAD:
                break # 如果在处理当前页时达到上限，也停止

             observation_id = obs.get('id')
             photos = obs.get('photos', [])

             if not photos:
                 continue # 跳过没有照片的记录 (虽然我们加了 photos=true 参数，但以防万一)

             for photo in photos:
                 photo_id = photo.get('id')
                 photo_url_template = photo.get('url')

                 if not photo_id or not photo_url_template:
                     continue # 跳过信息不完整的照片

                 # 构建指定尺寸的图片 URL
                 # iNaturalist URL 格式通常是 .../{size}.{ext}
                 # 我们替换掉 URL 模板中的尺寸部分
                 img_url = photo_url_template.replace('square', IMG_SIZE)

                 # 构建保存文件的路径
                 file_extension = os.path.splitext(img_url)[1].split('?')[0] # 获取 .jpg, .png 等，并移除可能的查询参数
                 if not file_extension: # 如果无法确定扩展名，默认使用 .jpg
                     file_extension = '.jpg'
                 filename = f"obs_{observation_id}_photo_{photo_id}{file_extension}"
                 filepath = os.path.join(output_dir, filename)

                 # 检查文件是否已存在，避免重复下载
                 if not os.path.exists(filepath):
                     print(f"  准备下载: {filename} (来自 {img_url})")
                     if download_image(img_url, filepath):
                         downloaded_count += 1
                         # 打印进度
                         if MAX_IMAGES_TO_DOWNLOAD:
                              print(f"  进度: {downloaded_count}/{MAX_IMAGES_TO_DOWNLOAD}")
                         else:
                              print(f"  已下载: {downloaded_count} 张")

                         # 检查是否达到下载上限
                         if MAX_IMAGES_TO_DOWNLOAD is not None and downloaded_count >= MAX_IMAGES_TO_DOWNLOAD:
                             break # 停止下载内部照片循环
                 else:
                     # print(f"  文件已存在，跳过: {filename}")
                     pass # 如果文件存在，可以取消注释上面的打印语句来查看信息

        # 准备获取下一页
        page += 1


    print("-" * 30)
    print(f"下载完成。总共下载了 {downloaded_count} 张图片到 '{output_dir}'。")

# --- 脚本入口 ---
if __name__ == "__main__":
    # --- 用户需要修改的部分 ---
    target_taxon_id = 1144756 # 示例：帝王蝶 (Danaus plexippus) 的 Taxon ID

    destination_folder = "inaturalist_images" # 图片保存的文件夹名称
    # 调用主函数执行下载
    download_inaturalist_images_by_taxon(target_taxon_id, destination_folder)

开始为 Taxon ID: 1144756 下载图片...
图片将保存到: inaturalist_images
图片尺寸: large
最多下载: 2000 张图片
------------------------------

正在获取第 1 页数据...
  找到总计约 17822 条带照片的观察记录。
  预计需要获取 10 页数据。
  处理 200 条观察记录...
  准备下载: obs_272085056_photo_489479506.jpg (来自 https://inaturalist-open-data.s3.amazonaws.com/photos/489479506/large.jpg)
  进度: 1/2000
  准备下载: obs_272080339_photo_489470775.jpeg (来自 https://inaturalist-open-data.s3.amazonaws.com/photos/489470775/large.jpeg)
  进度: 2/2000
  准备下载: obs_272080339_photo_489470811.jpeg (来自 https://inaturalist-open-data.s3.amazonaws.com/photos/489470811/large.jpeg)
  进度: 3/2000
  准备下载: obs_272080339_photo_489470968.jpeg (来自 https://inaturalist-open-data.s3.amazonaws.com/photos/489470968/large.jpeg)
  进度: 4/2000
  准备下载: obs_272080339_photo_489471113.jpeg (来自 https://inaturalist-open-data.s3.amazonaws.com/photos/489471113/large.jpeg)
  进度: 5/2000
  准备下载: obs_272057294_photo_489427277.jpg (来自 https://inaturalist-open-data.s3.amazonaws.com/photos/489427277/large.jpg)
  进度: 6/20